In [1]:
import pandas as pd
from dataloader import TextLoader
pd.set_option('display.max_colwidth', None)

In [2]:
train = pd.read_pickle("../../data/interim/cv_train.pickle");

train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,viretual_time_stamp
0,32933156,0,705741139,128,0,0,0,1,NaN,<NA>,0
1,32933157,20666,705741139,7860,0,1,0,1,16000.0,False,20666
2,32933158,39172,705741139,7922,0,2,1,1,19000.0,False,39172
3,32933159,58207,705741139,156,0,3,2,1,17000.0,False,58207
4,32933160,75779,705741139,51,0,4,0,1,17000.0,False,75779


In [3]:
del train["prior_question_had_explanation"]
del train["prior_question_elapsed_time"]
del train["viretual_time_stamp"]

train = train[train.content_type_id == False]
del train["content_type_id"]
del train["row_id"]

last_timestamp = train.groupby("user_id")[["timestamp","user_id"]].tail(1).set_index("user_id", drop=True)["timestamp"].to_dict()
train.reset_index(drop=True, inplace=True) #Resetting the index frees memory



train["timestamp"] = train.groupby("user_id")["timestamp"].diff().fillna(0)/1000
train["timestamp"] = train.timestamp.astype("int32")

boundaries = [120,600,1800,3600,10800,43200,86400,259200,604800]
x = train.timestamp.copy()

for i, boundary in enumerate(boundaries):
    
    if i == 0:
        start = 60
    else:
        start = boundaries[i-1]
        
    end = boundary
    
    train.loc[(x >= start) & (x < end), "timestamp"] = i+60
    
train.loc[x >= end, "timestamp"] = i+60+1

del x
train["timestamp"] = train["timestamp"].astype("int8")

train.head()

,timestamp,user_id,content_id,task_container_id,user_answer,answered_correctly
0,0,705741139,128,0,0,1
1,20,705741139,7860,1,0,1
2,18,705741139,7922,2,1,1
3,19,705741139,156,3,2,1
4,17,705741139,51,4,0,1


### Data Explanation

<p>user_id</p>(<br />
&nbsp;&nbsp;<b>[content_id]:</b> ID code for the user interaction<br />
&nbsp;&nbsp;<b>[answered_correctly]:</b> If the user responded correctly. Read -1 as null, for lecture<br />
&nbsp;&nbsp;<b>[timestamp]:</b> The difference of time between every two questions in seconds<br />
&nbsp;&nbsp;<b>[user_answer]:</b> The user's answer to the question, if any. Read -1 as null, for lectures.<br />
)

In [4]:
valid = pd.read_pickle("data/processed/validation.pickle")

valid.head()

user_id
1868605503    ([5612, 4259, 9261, 325, 296, 1314, 306, 1018, 444, 9261, 5612, 648, 33, 118, 51, 10590, 7898, 7914, 7875, 10527, 7912, 11, 183, 10657, 7, 7869, 7970, 8494, 8197, 4718, 9913, 4002, 5158, 4021, 8978, 9260, 6423, 5259, 9571, 9289, 5327, 6409, 8940, 4528, 8472, 4484, 8735, 4534, 5357, 3858, 8508, 6246, 6438, 5218, 4528, 5218, 9182, 3939, 3852, 6018, 9028, 5324, 5590, 4762, 8957, 4217, 5507, 5507, 4484, 4772, 9966, 5911, 4148, 5214, 5765, 6365, 4207, 3927, 8794, 5615, 4566, 3878, 8788, 3781, 6181, 4704, 9753, 4114, 4217, 5341, 6669, 8358, 4700, 5674, 5993, 8542, 4812, 4224, 5111, 8770], [0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1], [0, 27, 16, 28, 19, 22, 34, 62, 41, 63, 34, 35, 64, 31, 29, 25, 60, 60, 27, 29, 28, 36

In [12]:
questions = pd.read_csv("../../data/raw/questions.csv")

part_valus = questions.part.values

questions.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [9]:
val_group = pd.read_pickle("../../data/processed/validation.pickle")

valset = TextLoader(val_group)

valset.__getitem__(0)

(tensor([ 5612,  4259,  9261,   325,   296,  1314,   306,  1018,   444,  9261,
          5612,   648,    33,   118,    51, 10590,  7898,  7914,  7875, 10527,
          7912,    11,   183, 10657,     7,  7869,  7970,  8494,  8197,  4718,
          9913,  4002,  5158,  4021,  8978,  9260,  6423,  5259,  9571,  9289,
          5327,  6409,  8940,  4528,  8472,  4484,  8735,  4534,  5357,  3858,
          8508,  6246,  6438,  5218,  4528,  5218,  9182,  3939,  3852,  6018,
          9028,  5324,  5590,  4762,  8957,  4217,  5507,  5507,  4484,  4772,
          9966,  5911,  4148,  5214,  5765,  6365,  4207,  3927,  8794,  5615,
          4566,  3878,  8788,  3781,  6181,  4704,  9753,  4114,  4217,  5341,
          6669,  8358,  4700,  5674,  5993,  8542,  4812,  4224,  5111,  8770]),
 tensor([0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
         0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1

In [14]:
part_valus

array([1, 1, 1, ..., 5, 5, 5])

In [15]:
unique_ques = len(questions)

unique_ques

13523